# upscale the image

In [1]:
%pip install -q git+https://github.com/huggingface/transformers.git

Note: you may need to restart the kernel to use updated packages.


# Define the model

In [2]:
from transformers import Swin2SRForImageSuperResolution

model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")

/home/littlefish/miniconda3/envs/genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import Swin2SRImageProcessor 

processor = Swin2SRImageProcessor()

# Load the Image folder

In [4]:
import os
from PIL import Image
import numpy as np
import torch

In [ ]:
road_dir = "./pytorch-CycleGAN-and-pix2pix/results/ROAD_pix2pix/test_latest/submission"
river_dir = "./pytorch-CycleGAN-and-pix2pix/results/RIVER_pix2pix/test_latest/submission"
output_dir = "upscaled_images"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# road
for img_name in os.listdir(road_dir):
    img_path = os.path.join(road_dir, img_name)
    if img_name in os.listdir(output_dir):
        continue
    img = Image.open(img_path)
    pixel_values = processor(img, return_tensors="pt").pixel_values
    with torch.no_grad():
        outputs = model(pixel_values)
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    # resize to 420x240
    output = np.array(Image.fromarray(output).resize((420, 240), Image.BICUBIC))
    # save the image
    output_img = Image.fromarray(output)
    output_img.save(os.path.join(output_dir, img_name))
print("Upscaling road done!")

# river
for img_name in os.listdir(river_dir):
    img_path = os.path.join(river_dir, img_name)
    img = Image.open(img_path)
    if img_name in os.listdir(output_dir):
        continue
    pixel_values = processor(img, return_tensors="pt").pixel_values
    with torch.no_grad():
        outputs = model(pixel_values)
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    # resize to 420x240
    output = np.array(Image.fromarray(output).resize((420, 240), Image.BICUBIC))
    # save the image
    output_img = Image.fromarray(output)
    output_img.save(os.path.join(output_dir, img_name))
print("Upscaling river done!")

In [1]:
output_dir = "upscaled_images"

In [2]:
import os
from PIL import Image
# resize all the images in the output directory into 428x240

for img_name in os.listdir(output_dir):
    img_path = os.path.join(output_dir, img_name)
    img = Image.open(img_path)
    img = img.resize((428, 240), Image.BICUBIC)
    img.save(img_path)

In [5]:
print(f"Size: {len(os.listdir(output_dir))}")

Size: 1440


In [4]:
# zip the output
import shutil

shutil.make_archive(output_dir, 'zip', output_dir)

'/home/littlefish/gen-ai-uav/upscaled_images.zip'